In [1]:
import gspread
import pandas as pd
import requests
from oauth2client.service_account import ServiceAccountCredentials

# 🔹 Step 1: Google Sheets Setup
SHEET_ID = "1OR6GLerjtoky17YFVomuyXaEYFgSrUcf14Hwc0WCH2I"
TRAFFIC_SHEET_NAME = "TrafficData"
IP_SHEET_NAME = "IPData"

# Google Sheets API Authentication
json_keyfile = "/content/trafffic-maps-519d7d5c30ca.json"  # Update with your file path
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
client = gspread.authorize(creds)

# Open Spreadsheet
spreadsheet = client.open_by_key(SHEET_ID)

# Ensure "TrafficData" sheet exists
try:
    traffic_sheet = spreadsheet.worksheet(TRAFFIC_SHEET_NAME)
except gspread.exceptions.WorksheetNotFound:
    traffic_sheet = spreadsheet.add_worksheet(title=TRAFFIC_SHEET_NAME, rows="1000", cols="20")

# Ensure "IPData" sheet exists
try:
    ip_sheet = spreadsheet.worksheet(IP_SHEET_NAME)
except gspread.exceptions.WorksheetNotFound:
    ip_sheet = spreadsheet.add_worksheet(title=IP_SHEET_NAME, rows="100", cols="5")

# 🔹 Step 2: Define API Keys
IP_API_KEY = "e7e6bdecb7c54177bd0c05c180d7719b"
ROUTING_API_KEY = "6ca1a34284a841ca8eccb88f942f4ef9"
PLACES_API_KEY = "a3f06fd937fa4198b0c687197e34ced8"
GEOCODING_API_KEY = "cbc060bbc7664bf7849cf531abe1d41d"

# 🔹 Step 3: Define API URLs
ip_api_url = f"https://api.geoapify.com/v1/ipinfo?apiKey={IP_API_KEY}"
routing_url = f"https://api.geoapify.com/v1/routing?waypoints=50.9621,4.4145|50.4291,5.0008&mode=drive&apiKey={ROUTING_API_KEY}"
places_url = f"https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect:10.7164,48.7551,10.8353,48.6809&limit=50&apiKey={PLACES_API_KEY}"
geocode_url = f"https://api.geoapify.com/v1/geocode/reverse?lat=51.2171&lon=6.7782&apiKey={GEOCODING_API_KEY}"

# 🔹 Step 4: Fetch Data from APIs
try:
    ip_response = requests.get(ip_api_url).json()
    route_response = requests.get(routing_url).json()
    places_response = requests.get(places_url).json()
    geocode_response = requests.get(geocode_url).json()
except requests.exceptions.RequestException as e:
    print(f"❌ API request failed: {e}")
    exit()

# 🔹 Step 5: Extract IP Geolocation Data
ip_data = [["IP Address", "Country", "City", "IP_Latitude", "IP_Longitude"]]  # Headers

if "ip" in ip_response:
    ip_data.append([
        ip_response.get("ip", "N/A"),
        ip_response.get("country", {}).get("name", "N/A"),
        ip_response.get("city", {}).get("name", "N/A"),
        ip_response.get("location", {}).get("latitude", "N/A"),
        ip_response.get("location", {}).get("longitude", "N/A")
    ])

# 🔹 Step 6: Extract Route Data
route_data = [["Mode", "Distance (m)", "Time (s)", "Instruction", "Step Distance (m)", "Step Time (s)"]]
if "features" in route_response:
    for feature in route_response["features"]:
        props = feature["properties"]
        for leg in props.get("legs", []):
            for step in leg.get("steps", []):
                route_data.append([
                    props["mode"],
                    props["distance"],
                    props["time"],
                    step.get("instruction", {}).get("text", "N/A"),
                    step.get("distance", "N/A"),
                    step.get("time", "N/A")
                ])

# 🔹 Step 7: Extract Places Data
places_data = [["Name", "Categories", "Address", "Latitude", "Longitude"]]
if "features" in places_response:
    for place in places_response["features"]:
        props = place["properties"]
        places_data.append([
            props.get("name", "Unknown"),
            ", ".join(props.get("categories", [])),
            props.get("address_line1", ""),
            props.get("lat", ""),
            props.get("lon", "")
        ])

# 🔹 Step 8: Extract Geocode Data
geocode_data = [["Formatted Address", "Geo_Latitude", "Geo_Longitude"]]
if "features" in geocode_response:
    for place in geocode_response["features"]:
        props = place["properties"]
        geocode_data.append([
            props.get("formatted", ""),
            props.get("lat", ""),
            props.get("lon", "")
        ])

# 🔹 Step 9: Match and Combine Data
combined_data = [["Name", "Categories", "Address", "Latitude", "Longitude",
                  "Mode", "Distance (m)", "Time (s)", "Instruction", "Step Distance (m)", "Step Time (s)",
                  "Formatted Address", "Geo_Latitude", "Geo_Longitude"]]

max_rows = max(len(places_data), len(route_data), len(geocode_data))

for i in range(1, max_rows):  # Skip headers
    place_info = places_data[i] if i < len(places_data) else ["N/A"] * 5
    route_info = route_data[i] if i < len(route_data) else ["N/A"] * 6
    geocode_info = geocode_data[i] if i < len(geocode_data) else ["N/A"] * 3
    combined_data.append(place_info + route_info + geocode_info)

# 🔹 Step 10: Upload Data to Google Sheets
# ✅ Update Traffic Data
traffic_sheet.clear()
traffic_sheet.append_rows(combined_data)

# ✅ Update IP Geolocation Data
ip_sheet.clear()
ip_sheet.append_rows(ip_data)

print("✅ Data updated successfully in Google Sheets!")


✅ Data updated successfully in Google Sheets!
